In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

### 1. Guardar Archivos para El Diente y El Cuchillo

In [2]:
def read_csv_files(directory):

    data_dict = {}
    files = os.listdir(directory)
    
    for file in files:
        if file.endswith('.csv'):
            filename = file.split('_', 1)[-1]
            filename = filename.split('.')[0]
            
            file_path = os.path.join(directory, file)
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(file_path , encoding='latin-1')
            # Store the DataFrame in the dictionary
            data_dict[filename] = df
            
    return data_dict

In [3]:
directory = "../data/climatologia/EMAs/el_cuchillo/csv"
cuchillo = read_csv_files(directory)

In [66]:
directory = "../data/climatologia/EMAs/el_diente/csv"
diente = read_csv_files(directory)

### 2. Estandarizar columnas

In [4]:
def replace_empty_strings_with_nan(data_dict):
    replaced_data_dict = {}
    
    for filename, df in data_dict.items():
        # Replace empty strings with NaN
        replaced_df = df.replace("", np.nan)
        replaced_df = df.replace(' ', np.nan)
        replaced_data_dict[filename] = replaced_df
    
    return replaced_data_dict

In [5]:
def clean_data_first_row(data_dict):
    
    cleaned_data_dict = {}
    
    for filename, df in data_dict.items():
        # Check if the value in [0,0] is empty
        if pd.isna(df.iloc[0, 0]):
            # Drop the first row
            cleaned_df = df.drop(index=0)
        else:
            cleaned_df = df
            
        cleaned_data_dict[filename] = cleaned_df
    
    return cleaned_data_dict

In [6]:
cuchillo = replace_empty_strings_with_nan(cuchillo)
cuchillo = clean_data_first_row(cuchillo)

In [7]:
def get_unique_column_names(data_dict):
    unique_column_names = set()
    
    for df in data_dict.values():
        unique_column_names.update(df.columns)
    
    return list(unique_column_names)

In [8]:
unique_column_names = get_unique_column_names(cuchillo)
unique_column_names

['Time',
 'Precipitacion',
 'Date',
 'fecha',
 'TempAire',
 'Rapidez de ráfaga (km/h)',
 'PresBarometric',
 'Dirección de ráfaga (grados)',
 'WSMK',
 'WSK',
 'DirViento',
 'Rapidez de viento (km/h)',
 'WSMDir',
 'Presión Atmosférica (hpa)',
 'Rain',
 'Precipitación (mm)',
 'AvgTemp',
 'Radiación Solar (W/m²)',
 'RapRafaga',
 'AvgSR',
 'AvgRh',
 'RapViento',
 'DirRafaga',
 'nombre_estacion',
 'Temperatura del Aire (°C)',
 'HumRelativa',
 'Fecha Local',
 'Fecha UTC',
 'Dir',
 'Humedad relativa (%)',
 'AvgBP',
 'RadSolar',
 'Dirección del Viento (grados)']

In [9]:
def drop_columns(data_dict, columns_to_drop):
    modified_data_dict = {}
    
    for filename, df in data_dict.items():
        modified_df = df.drop(columns=columns_to_drop, errors='ignore')
        modified_data_dict[filename] = modified_df
    
    return modified_data_dict

In [10]:
columns_to_drop = ['Fecha UTC', 'nombre_estacion']  # Specify columns to drop
cuchillo = drop_columns(cuchillo, columns_to_drop)

In [11]:
def rename_columns(data_dict, column_dict):
    modified_data_dict = {}
    
    for filename, df in data_dict.items():
        renamed_columns = {}
        for original_column, new_column in column_dict.items():
            for col in df.columns:
                if col in column_dict[original_column]:
                    renamed_columns[col] = original_column

        renamed_df = df.rename(columns=renamed_columns)
        modified_data_dict[filename] = renamed_df
    
    return modified_data_dict

In [12]:
column_dict = {'Direccion del viento (g)' : ['DirViento','Dir','Dirección del Viento (grados)'],
               'Direccion de rafaga (g)' : ['DirRafaga','Dirección de ráfaga (grados)','WSMDir'],
               'Rapidez de viento (km/h)' : ['WSK','Rapidez de viento (km/h)','RapViento'],
               'Rapidez de rafaga (km/h)' : ['RapRafaga','WSMK','Rapidez de ráfaga (km/h)'],
               'Temperatura (C)' : ['AvgTemp','Temperatura del Aire (°C)','TempAire'],
               'Humedad Relativa (%)' : ['HumRelativa','AvgRh','Humedad relativa (%)'],
               'Presion Atmosferica (hpa)' : ['PresBarometric','Presión Atmosférica (hpa)','AvgBP'],
               'Precipitacion (mm)' : ['Precipitación (mm)', 'Precipitacion','Rain'],
               'Radiacion Solar (W/m2)' : ['AvgSR','Radiación Solar (W/m²)','RadSolar']
              }

In [13]:
cuchillo = rename_columns(cuchillo, column_dict)
unique_column_names = get_unique_column_names(cuchillo)
unique_column_names

['Time',
 'Humedad Relativa (%)',
 'Date',
 'Precipitacion (mm)',
 'Direccion del viento (g)',
 'Rapidez de viento (km/h)',
 'fecha',
 'Fecha Local',
 'Temperatura (C)',
 'Presion Atmosferica (hpa)',
 'Rapidez de rafaga (km/h)',
 'Radiacion Solar (W/m2)',
 'Direccion de rafaga (g)']

In [14]:
cuchillo['2007_01'].head()

,Date,Time,Direccion del viento (g),Direccion de rafaga (g),Rapidez de viento (km/h),Rapidez de rafaga (km/h),Temperatura (C),Humedad Relativa (%),Presion Atmosferica (hpa),Precipitacion (mm),Radiacion Solar (W/m2)
1,2007 Ene 01,0:00,34,33,7.4,11.4,19.9,56,1003.9,0,1
2,2007 Ene 01,0:10,42,63,6,8.5,19.4,58,1004,0,-3
3,2007 Ene 01,0:20,47,66,4.6,12,18.8,60,1004.1,0,-5
4,2007 Ene 01,0:30,45,53,5.1,7.9,18.5,61,1004.2,0,-5
5,2007 Ene 01,0:40,32,6,5.5,7.9,18,63,1004.3,0,-5


### 2.1 Limpiar formatos de columnas de interes

In [78]:
def clean_data(data_dict):
    
    for filename, df in data_dict.items():
        df['Temperatura (C)'] = pd.to_numeric(df['Temperatura (C)'], errors='coerce')
        df['Precipitacion (mm)'] = pd.to_numeric(df['Precipitacion (mm)'], errors='coerce')
    
    return data_dict

In [79]:
cuchillo = clean_data(cuchillo)

### 3. Estandarizar Fechas

In [15]:
cuchillo['2023_05'].head() 

,Fecha Local,Direccion del viento (g),Direccion de rafaga (g),Rapidez de viento (km/h),Rapidez de rafaga (km/h),Temperatura (C),Humedad Relativa (%),Presion Atmosferica (hpa),Precipitacion (mm),Radiacion Solar (W/m2)
0,5/1/2023 0:00,134,126,0,0.0,23.8,45,994.1,0,0
1,5/1/2023 0:10,147,175,0,0.0,23.6,45,994.0,0,0
2,5/1/2023 0:20,136,128,0,0.0,23.3,46,993.9,0,0
3,5/1/2023 0:30,132,168,0,0.0,23.1,46,993.7,0,0
4,5/1/2023 0:40,122,98,0,0.0,22.9,46,993.5,0,0


In [59]:
def format_datetime_columns(data_dict):

    formatted_data_dict = {}
    original_format = "%m/%d/%Y %H:%M"  # Original datetime format
    iso8601_format = "%Y-%m-%d %H:%M:%S"  # ISO 8601 format

    for filename, df in data_dict.items():
        if 'fecha' in df.columns:
            df['fecha'] = pd.to_datetime(df['fecha'], format=original_format).dt.strftime(iso8601_format)
        if 'Fecha Local' in df.columns:
            df['Fecha Local'] = pd.to_datetime(df['Fecha Local'], format=original_format).dt.strftime(iso8601_format)
        
        formatted_data_dict[filename] = df
    
    return formatted_data_dict

In [20]:
cuchillo = format_datetime_columns(cuchillo)

In [21]:
cuchillo['2023_05'].head() 

,Fecha Local,Direccion del viento (g),Direccion de rafaga (g),Rapidez de viento (km/h),Rapidez de rafaga (km/h),Temperatura (C),Humedad Relativa (%),Presion Atmosferica (hpa),Precipitacion (mm),Radiacion Solar (W/m2)
0,2023-05-01 00:00:00,134,126,0,0.0,23.8,45,994.1,0,0
1,2023-05-01 00:10:00,147,175,0,0.0,23.6,45,994.0,0,0
2,2023-05-01 00:20:00,136,128,0,0.0,23.3,46,993.9,0,0
3,2023-05-01 00:30:00,132,168,0,0.0,23.1,46,993.7,0,0
4,2023-05-01 00:40:00,122,98,0,0.0,22.9,46,993.5,0,0


### 3.1 Corregir formato de fechas con abreviaturas

In [22]:
def replace_month_strings(data_dict):

    month_mapping = {
        " Ene ": "-01-",
        " Feb ": "-02-",
        " Mar ": "-03-",
        " Abr ": "-04-",
        " May ": "-05-",
        " Jun ": "-06-",
        " Jul ": "-07-",
        " Ago ": "-08-",
        " Sep ": "-09-",
        " Oct ": "-10-",
        " Nov ": "-11-",
        " Dic ": "-12-",
        " ene ": "-01-",
        " feb ": "-02-",
        " mar ": "-03-",
        " abr ": "-04-",
        " may ": "-05-",
        " jun ": "-06-",
        " jul ": "-07-",
        " ago ": "-08-",
        " sep ": "-09-",
        " oct ": "-10-",
        " nov ": "-11-",
        " dic ": "-12-",
    }

    # Iterate through each DataFrame in the dictionary
    for filename, df in data_dict.items():
        # Check if 'Date' column exists in the DataFrame
        if 'Date' in df.columns:
            # Replace month abbreviations with numerical values
            for old_month, new_month in month_mapping.items():
                df['Date'] = df['Date'].str.replace(old_month, new_month)

    return data_dict

In [23]:
cuchillo = replace_month_strings(cuchillo)

In [24]:
def create_iso8601_column(data_dict):

    formatted_data_dict = {}

    for filename, df in data_dict.items():
        # Check if both 'Date' and 'Time' columns exist in the DataFrame
        if 'Date' in df.columns and 'Time' in df.columns:
            # Convert 'Date' column to datetime objects
            df['Date'] = pd.to_datetime(df['Date'])
            # Convert 'Time' column to timedelta objects
            df['Time'] = pd.to_timedelta(df['Time'] + ':00')
            # Combine 'Date' and 'Time' columns into a single datetime column
            df['fecha nueva'] = df['Date'] + df['Time']
            # Convert the datetime column to ISO 8601 format
            df['fecha nueva'] = df['fecha nueva'].dt.strftime('%Y-%m-%d %H:%M:%S')
            # Drop the original 'Date' and 'Time' columns
            df.drop(columns=['Date', 'Time'], inplace=True)
            # Insert the 'fecha nueva' column as the first column
            df.insert(0, 'fecha nueva', df.pop('fecha nueva'))
        
        formatted_data_dict[filename] = df
    
    return formatted_data_dict

In [25]:
cuchillo = create_iso8601_column(cuchillo)

In [26]:
cuchillo['2000_04'].head()

,fecha nueva,Direccion del viento (g),Direccion de rafaga (g),Rapidez de viento (km/h),Rapidez de rafaga (km/h),Temperatura (C),Humedad Relativa (%),Presion Atmosferica (hpa),Precipitacion (mm),Radiacion Solar (W/m2)
0,2000-04-10 12:10:00,259,246,9.7,17.2,31.8,34,991.6,0.00,655
1,2000-04-10 20:20:00,260,288,8.0,16.0,31.9,34,NaN,2.54,NaN
2,2000-04-10 20:30:00,NaN,288,0.0,0.0,32.2,34,NaN,0.00,NaN
3,2000-04-10 20:40:00,NaN,288,0.0,0.0,32.7,33,NaN,0.00,NaN
4,2000-04-10 20:50:00,148,155,3.1,11.4,33.3,32,NaN,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...
2747,2000-04-30 23:10:00,114,106,25.6,35.6,35,35,988.3,0.00,152
2748,2000-04-30 23:20:00,109,106,27.6,39.6,34.8,37,988.5,0.00,123
2749,2000-04-30 23:30:00,114,118,27.6,37.9,34.3,39,988.7,0.00,95
2750,2000-04-30 23:40:00,111,100,29.2,36.7,33.8,42,988.8,0.00,78


In [27]:
# Renombrar las columnas de fechas
column_date_dict = {'Fecha' : ['fecha','Fecha Local','fecha nueva']}
cuchillo = rename_columns(cuchillo, column_date_dict)
unique_column_names = get_unique_column_names(cuchillo)
unique_column_names

### 3.2 Separar la columna de Fecha en fecha y hora para la construccion del nuevo dataframe

In [28]:
def split_fecha_column(data_dict):

    formatted_data_dict = {}

    for filename, df in data_dict.items():
        # Split 'Fecha' column into 'Fecha' and 'Hora' columns
        df[['Fecha', 'Hora']] = df['Fecha'].str.split(' ', expand=True)
        # Convert 'Hora' column to timedelta objects
        df['Hora'] = pd.to_timedelta(df['Hora'])
        # Convert timedelta objects to string with format 'HH:MM'
        df['Hora'] = df['Hora'].dt.components['hours'].astype(str).str.zfill(2) + ':' + df['Hora'].dt.components['minutes'].astype(str).str.zfill(2)
        # Insert 'Hora' column as the second column
        df.insert(1, 'Hora', df.pop('Hora'))
        # Ensure 'Fecha' column is datetime object
        df['Fecha'] = pd.to_datetime(df['Fecha'])
        # Reorder columns so 'Fecha' and 'Hora' are the first and second columns
        df = df[['Fecha', 'Hora'] + [col for col in df.columns if col not in ['Fecha', 'Hora']]]
        
        formatted_data_dict[filename] = df
    
    return formatted_data_dict

In [29]:
cuchillo = split_fecha_column(cuchillo)

In [30]:
cuchillo['2000_04'].head()

,Fecha,Hora,Direccion del viento (g),Direccion de rafaga (g),Rapidez de viento (km/h),Rapidez de rafaga (km/h),Temperatura (C),Humedad Relativa (%),Presion Atmosferica (hpa),Precipitacion (mm),Radiacion Solar (W/m2)
0,2000-04-10,12:10,259,246,9.7,17.2,31.8,34,991.6,0.00,655
1,2000-04-10,20:20,260,288,8.0,16.0,31.9,34,NaN,2.54,NaN
2,2000-04-10,20:30,NaN,288,0.0,0.0,32.2,34,NaN,0.00,NaN
3,2000-04-10,20:40,NaN,288,0.0,0.0,32.7,33,NaN,0.00,NaN
4,2000-04-10,20:50,148,155,3.1,11.4,33.3,32,NaN,0.00,NaN


### 4. Ahora hacer lo mismo para los datos de El Diente

In [67]:
diente = replace_empty_strings_with_nan(diente)
diente = clean_data_first_row(diente)
unique_column_names = get_unique_column_names(diente)
unique_column_names

['Precipitacion',
 'fecha',
 'TempAire',
 'Rapidez de ráfaga (km/h)',
 'PresBarometric',
 'Dirección de ráfaga (grados)',
 'TempComb',
 'Rapidez de viento (km/h)',
 'DirViento',
 'Presión Atmosférica (hpa)',
 'Precipitación (mm)',
 'Radiación Solar (W/m²)',
 'RapRafaga',
 'RapViento',
 'DirRafaga',
 'nombre_estacion',
 'Temperatura del Aire (°C)',
 'HumRelativa',
 'Fecha Local',
 'Fecha UTC',
 'Humedad relativa (%)',
 'RadSolar',
 'Dirección del Viento (grados)']

In [68]:
columns_to_drop = ['nombre_estacion', 'TempComb', 'Fecha UTC']  # Specify columns to drop
diente = drop_columns(diente, columns_to_drop)
diente = rename_columns(diente, column_dict)
unique_column_names = get_unique_column_names(diente)
unique_column_names

['Humedad Relativa (%)',
 'Precipitacion (mm)',
 'Direccion del viento (g)',
 'fecha',
 'Rapidez de viento (km/h)',
 'Fecha Local',
 'Temperatura (C)',
 'Presion Atmosferica (hpa)',
 'Rapidez de rafaga (km/h)',
 'Radiacion Solar (W/m2)',
 'Direccion de rafaga (g)']

In [69]:
diente = format_datetime_columns(diente)
column_date_dict = {'Fecha' : ['fecha','Fecha Local']}
diente = rename_columns(diente, column_date_dict)

In [70]:
unique_column_names = get_unique_column_names(diente)
unique_column_names

['Humedad Relativa (%)',
 'Precipitacion (mm)',
 'Direccion del viento (g)',
 'Rapidez de viento (km/h)',
 'Temperatura (C)',
 'Presion Atmosferica (hpa)',
 'Rapidez de rafaga (km/h)',
 'Fecha',
 'Radiacion Solar (W/m2)',
 'Direccion de rafaga (g)']

In [71]:
diente['2022_05'].head()

,Fecha,Direccion del viento (g),Direccion de rafaga (g),Rapidez de viento (km/h),Rapidez de rafaga (km/h),Temperatura (C),Humedad Relativa (%),Presion Atmosferica (hpa),Precipitacion (mm),Radiacion Solar (W/m2)
0,2022-05-01 00:00:00,15,49,8.7,21.2,29.7,58,923.9,0.0,70
1,2022-05-01 00:10:00,348,348,11.2,25.9,29.5,59,923.9,0.0,60
2,2022-05-01 00:20:00,7,354,10.4,22.3,29.3,60,923.9,0.0,57
3,2022-05-01 00:30:00,16,6,13.5,27.0,29.2,62,924.0,0.0,55
4,2022-05-01 00:40:00,9,347,10.6,24.8,29.1,62,924.1,0.0,32


In [72]:
diente = split_fecha_column(diente)

In [85]:
diente = clean_data(diente)

In [73]:
diente['2022_06'].head()

,Fecha,Hora,Direccion del viento (g),Direccion de rafaga (g),Rapidez de viento (km/h),Rapidez de rafaga (km/h),Temperatura (C),Humedad Relativa (%),Presion Atmosferica (hpa),Precipitacion (mm),Radiacion Solar (W/m2)
0,2022-05-31,19:00,19,13,14.5,30.2,33.0,41,922.9,0.0,87
1,2022-05-31,19:10,8,26,15.1,31.7,33.0,41,922.9,0.0,86
2,2022-05-31,19:20,5,351,12.0,31.0,33.1,43,923.0,0.0,71
3,2022-05-31,19:30,19,52,14.6,32.8,32.9,48,923.2,0.0,53
4,2022-05-31,19:40,12,14,17.4,38.9,32.2,56,923.4,0.0,39


In [58]:
def find_dataframes_without_hora_column(data_dict):
    dataframes_without_hora_column = []

    for filename, df in data_dict.items():
        if 'Hora' not in df.columns:
            dataframes_without_hora_column.append(filename)
    
    return dataframes_without_hora_column

# Example usage:
dataframes_without_hora_column = find_dataframes_without_hora_column(diente)
print("DataFrames without 'Hora' column:", dataframes_without_hora_column)


DataFrames without 'Hora' column: ['2022_06', '2022_07', '2022_08', '2022_09', '2022_10', '2022_11', '2022_12', '2023_01', '2023_02', '2023_03', '2023_04', '2023_05', '2023_06', '2023_07', '2023_08', '2023_09', '2023_10', '2023_11', '2023_12', '2024_01']


### 5. Crear Dataframe de consulta de Precipitacion y Temperatura

In [29]:
export_data = pd.DataFrame(columns=['Fecha', 'Precipitacion (mm)', 'T_Max (C)', 'T_Min (C)'])

In [82]:
def create_daily_aggregate_dataframe(data_dict):
    daily_aggregate_data = []

    # Iterate through each DataFrame in the dictionary
    for df in data_dict.values():
        # Iterate through each unique date in the 'Fecha' column
        for date in df['Fecha'].unique():
            # Filter the DataFrame for the current date
            date_data = df[df['Fecha'] == date]
            # Calculate total precipitation, maximum temperature, and minimum temperature for the current date
            precip_sum = date_data['Precipitacion (mm)'].sum()
            max_temp = date_data['Temperatura (C)'].max()
            min_temp = date_data['Temperatura (C)'].min()
            # Append a row to the new DataFrame with the aggregated values
            daily_aggregate_data.append({'Fecha': date,
                                          'Precipitacion (mm)': precip_sum,
                                          'T_Max (C)': max_temp,
                                          'T_Min (C)': min_temp})

    # Convert list of dictionaries to DataFrame
    daily_aggregate_data = pd.DataFrame(daily_aggregate_data)

    return daily_aggregate_data

In [88]:
# Example usage:
EMA_cuchillo_clean = create_daily_aggregate_dataframe(cuchillo)
EMA_cuchillo_clean.head()

,Fecha,Precipitacion (mm),T_Max (C),T_Min (C)
0,2000-04-10,2.54,34.4,31.8
1,2000-04-11,0.00,34.9,21.8
2,2000-04-12,49.51,29.4,15.8
3,2000-04-13,0.00,26.8,16.8
4,2000-04-14,0.00,31.5,17.7


In [89]:
EMA_diente_clean = create_daily_aggregate_dataframe(diente)
EMA_diente_clean.head()

,Fecha,Precipitacion (mm),T_Max (C),T_Min (C)
0,2013-01-01,0.0,NaN,NaN
1,2013-01-02,7.4,7.4,4.6
2,2013-01-03,0.2,5.9,1.2
3,2013-01-04,0.0,10.8,0.8
4,2013-01-05,0.0,18.5,4.3


### 5.1 Exportar

In [90]:
EMA_cuchillo_clean.to_csv('../data/clean_data/EMAs/EMA_cuchillo.csv', index=False)
EMA_diente_clean.to_csv('../data/clean_data/EMAs/EMA_diente.csv', index=False)